# ConvNet with MNIST

This tutorial uses some materials from the course CS 20SI: Tensorflow for Deep Learning Research, University of Stanford.

<img src='./resources/im13.png'/>

<img src='./resources/im14.png'/>

In [8]:
""" Using convolutional net on MNIST dataset of handwritten digit
(http://yann.lecun.com/exdb/mnist/)
"""
from __future__ import print_function

import os
import time 

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

N_CLASSES = 10

tf.reset_default_graph()

## Step 1: Read in data

In [2]:
# using TF Learn's built in function to load MNIST data to the folder [working_directory]/data/mnist
mnist = input_data.read_data_sets("data/mnist", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


## Step 2: Define parameters

In [3]:
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 10
DROPOUT = 0.75
N_EPOCHS = 50

## Step 3: Create placeholders for features and labels

In this section, there are few concepts that we need to know.

* **Variable sharing**. What if you want to manipulate on variables in different functions? You will pass those variables into functions as references. **Variable sharing mechanism** in TensorFlow allows for sharing variables accessed in different parts of the code without passing them as references. 

[tf.get_variable()](https://www.tensorflow.org/api_docs/python/tf/get_variable):  get existing variable.

* **Variable scope**. To enable **variable sharing**, TensorFlow allows to create different **scopes** in which variables are declared. Each scope can be interpreted as a **namespace**. A variable named **weights** is defined in scope named **conv1** will become **conv1/weights**

In [9]:
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# We'll be doing dropout for hidden layer so we'll need a placeholder
# for the dropout probability too
# Use None for shape so we can change the batch_size once we've built the graph
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, [None, 784], name="X_placeholder")
    Y = tf.placeholder(tf.float32, [None, 10], name="Y_placeholder")

dropout = tf.placeholder(tf.float32, name='dropout')

## Step 4 + 5: create weights + do inference

In [10]:
global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

Design the first conv layer.

In [11]:
with tf.variable_scope('conv1') as scope:
    # first, reshape the image to [BATCH_SIZE, 28, 28, 1] to make it work with tf.nn.conv2d
    # use the dynamic dimension -1
    X_reshape = tf.reshape(X, [BATCH_SIZE, 28, 28, 1], 'X_reshape')
    
    # create kernel variable of dimension [5, 5, 1, 32]
    # use tf.truncated_normal_initializer()
    kernel = tf.get_variable('kernels', shape=[5, 5, 1, 32],
                             initializer=tf.truncated_normal_initializer())

    # create biases variable of dimension [32]
    # use tf.random_normal_initializer()
    biases = tf.get_variable('biases', shape=[32],
                             initializer=tf.random_normal_initializer())
    
    # apply tf.nn.conv2d. strides [1, 1, 1, 1], padding is 'SAME'
    conv = tf.nn.conv2d(X_reshape, kernel, strides=[1, 1, 1, 1], padding='SAME')

    # apply relu on the sum of convolution output and biases
    conv1 = tf.nn.relu(conv + biases, name=scope.name)
    
    # output is of dimension BATCH_SIZE x 28 x 28 x 32
with tf.variable_scope('pool1') as scope:
    # apply max pool with ksize [1, 2, 2, 1], and strides [1, 2, 2, 1], padding 'SAME'
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # output is of dimension BATCH_SIZE x 14 x 14 x 32

(?, 784)
(128, 28, 28, 1)


Design the second conv layer which is similar to the first one, except the kernel size

In [ ]:
with tf.variable_scope('conv2') as scope:
    # similar to conv1, except kernel now is of the size 5 x 5 x 32 x 64
    kernel = tf.get_variable('kernels', shape=[5, 5, 32, 64], 
                        initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', shape=[64], 
                        initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(pool1, kernel, strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.relu(conv + biases, name=scope.name)

    # output is of dimension BATCH_SIZE x 14 x 14 x 64

with tf.variable_scope('pool2') as scope:
    # similar to pool1
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                            padding='SAME')

Create fully connected layer.

In [ ]:
with tf.variable_scope('fc') as scope:
    # use weight of dimension 7 * 7 * 64 x 1024
    input_features = 7 * 7 * 64
    
    # create weights and biases
    weights = tf.get_variable('weights', shape=[input_features, 1024], 
                             initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [1024], 
                        initializer=tf.random_normal_initializer())
    
    # reshape pool2 to 2 dimensional
    pool2 = tf.reshape(pool2, [-1, input_features])

    # apply relu on matmul of pool2 and w + b
    fc = tf.nn.relu(tf.matmul(pool2, weights) + biases)

    # apply dropout
    fc = tf.nn.dropout(fc, dropout, name='relu_dropout')
    
    # output is of dimension 1 x 1024

Apply soft max.

In [ ]:
with tf.variable_scope('softmax_linear') as scope:
    # this you should know. get logits without softmax
    # you need to create weights and biases
    weights = tf.get_variable('weights', shape=[1024, N_CLASSES], 
                             initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [N_CLASSES], 
                        initializer=tf.random_normal_initializer())
    logits = tf.matmul(fc, weights) + biases

## Step 6: Define loss function

In [ ]:
# use softmax cross entropy with logits as the loss function
# compute mean cross entropy, softmax is applied internally
with tf.name_scope('loss'):
    # you should know how to do this too
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    loss = tf.reduce_mean(entropy, name='loss')

## Step 7: Define training operation

In [ ]:
# Step 7: define training op
# using gradient descent with learning rate of LEARNING_RATE to minimize cost
# global_step is a variable to increment by one after the variables have been updated
optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss, 
                                        global_step=global_step)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    # to visualize using TensorBoard
    writer = tf.summary.FileWriter('./graphs/conv_mnist', sess.graph)
    
    ##### You have to create folders to store checkpoints
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/conv_mnist/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    
    initial_step = global_step.eval()
    start_time = time.time()
    n_batches = int(mnist.train.num_examples / BATCH_SIZE)
    
    total_loss = 0.0
    for index in range(initial_step, n_batches * N_EPOCHS): # train the model n_epochs times
        X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
        _, loss_batch = sess.run([optimizer, loss], 
                                feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT})
        total_loss += loss_batch
        if (index + 1) % SKIP_STEP == 0:
            print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
            total_loss = 0.0
            saver.save(sess, 'checkpoints/conv_mnist/checkpoint', index)
            
    print("Optimization Finished!") # should be around 0.35 after 25 epochs
    print("Total time: {0} seconds".format(time.time() - start_time))